In [ ]:
# put it under the folder MASA

In [1]:
import os
os.environ['CUDA_DEVICE_ORDRE'] = 'PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
import random 
import numpy as np
import torch as th
import datetime
import copy
th.backends.cudnn.deterministic = True
th.backends.cudnn.benchmark = False
os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':4096:8'
th.use_deterministic_algorithms(True)

import pandas as pd
import time
from config import Config
from utils.featGen import FeatureProcesser
from utils.tradeEnv import StockPortfolioEnv, StockPortfolioEnv_cash
from utils.model_pool import model_select, benchmark_algo_select
from utils.callback_func import PoCallback
from RL_controller.market_obs import MarketObserver, MarketObserver_Algorithmic
import timeit

from entrance import RLonly, RLcontroller, entrance
from config import Config
import optuna

In [2]:
# entrance()

In [3]:
def sample_params(trial: optuna.Trial):
    lr = trial.suggest_float("lr", 1e-5, 1e-2, log=True)
    tau = trial.suggest_float("tau", 1e-4, 1e-1, log=True)
    gamma = trial.suggest_float("gamma", 0.01, 0.99, log=False)
    trial_number = trial.number
    sampled_params = dict(lr=lr, tau=tau, gamma=gamma, trial_number=trial_number)
    return sampled_params

In [4]:
def my_entrance(sample_params, config=None):
    """
    Entrance function for running the MASA framework
    """   
    if not config:
        current_date = datetime.datetime.now().strftime('%Y-%m-%d-%H-%M-%S')
        rand_seed = int(time.mktime(datetime.datetime.strptime(current_date, '%Y-%m-%d-%H-%M-%S').timetuple()))

        random.seed(rand_seed)
        os.environ['PYTHONHASHSEED'] = str(rand_seed)
        np.random.seed(rand_seed)
        th.manual_seed(rand_seed)
        th.cuda.manual_seed(rand_seed)
        th.cuda.manual_seed_all(rand_seed)
        config = Config(seed_num=rand_seed, current_date=current_date) 
        
    start_cputime = time.process_time()
    start_systime = time.perf_counter()
        
    # replace with sampled params
    config.model_para['learning_rate'] = sample_params["lr"]
    config.model_para['tau'] = sample_params["tau"]
    config.model_para['gamma'] = sample_params["gamma"]
    
    config.print_config()
    if config.mode == 'RLonly':
        RLonly(config=config)
    elif config.mode == 'RLcontroller':
        RLcontroller(config=config)
    elif config.mode == 'Benchmark':
        raise NotImplementedError("Please refer to the corresponding papers and their provided implementations.")
    else:
        raise ValueError('Unexpected mode {}'.format(config.mode))

    end_cputime = time.process_time()
    end_systime = time.perf_counter()
    print("[Done] Total cputime: {} s, system time: {} s".format(np.round(end_cputime - start_cputime, 2), np.round(end_systime - start_systime, 2)))

In [5]:
def objective(trial: optuna.Trial) -> float:
    current_date = datetime.datetime.now().strftime('%Y-%m-%d-%H-%M-%S')
    rand_seed = int(time.mktime(datetime.datetime.strptime(current_date, '%Y-%m-%d-%H-%M-%S').timetuple()))
    # generate config
    random.seed(rand_seed)
    os.environ['PYTHONHASHSEED'] = str(rand_seed)
    np.random.seed(rand_seed)
    th.manual_seed(rand_seed)
    th.cuda.manual_seed(rand_seed)
    th.cuda.manual_seed_all(rand_seed)
    config = Config(seed_num=rand_seed, current_date=current_date) 
    # generate params
    sampled_params = sample_params(trial)   
    # run train and test
    my_entrance(sampled_params, config)
    # get loss
    file_name = os.path.join('res', 'RLcontroller', 'TD3', 'DJIA-10', current_date, 'test_bestmodel.csv')
    test_bestmodel = pd.read_csv(file_name)
    loss = test_bestmodel.loc[0]['js_loss']
    return loss

In [6]:
def finetune_hyperparameters(n_trials=5):
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=n_trials)
    print("Best trial:")
    trial = study.best_trial
    print("  Value: ", trial.value)
    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))
    print("  Best trial number: ", trial.number)
    my_entrance(trial.params)

In [7]:
finetune_hyperparameters(n_trials=15)

[I 2024-05-12 12:56:10,835] A new study created in memory with name: no-name-56696d93-f8a9-4b2e-a3e9-32d5aea96b81


AAMAS MASA Implementation 
mode: RLcontroller, rl_model_name: TD3, market_name: DJIA, topK: 10, dataDir: ./data, enable_controller: True, 
trade_pattern: 1 
period_mode: 1, num_epochs: 50, cov_lookback: 5, norm_method: sum, benchmark_algo: MASA-dc, trained_best_model_type: js_loss, pricePredModel: MA, 
is_enable_dynamic_risk_bound: True, risk_market: 0.001, risk_default: 0.017, cbf_gamma: 0.7, ars_trial: 10 
cur_datetime: 2024-05-12-12-56-10, res_dir: ./res\RLcontroller\TD3\DJIA-10\2024-05-12-12-56-10, tradeDays_per_year: 252, tradeDays_per_month: 21, seed_num: 1715532970, 
train_date_start: 2023-05-01 00:00:00, train_date_end: 2023-08-30 23:59:59, valid_date_start: 2023-08-31 00:00:00, valid_date_end: 2023-11-09 23:59:59, test_date_start: 2023-11-10 00:00:00, test_date_end: 2024-03-01 23:59:59, 
tech_indicator_input_lst: ['CHANGE'], 
otherRef_indicator_lst: ['MA-5', 'DAILYRETURNS-5'], enable_cov_features: False 
tmp_name: Cls3_RLcontroller_dc_1_K10_M1_DJIA_js_loss, mkt_rf: {'SP500': 1

[I 2024-05-12 12:57:10,169] Trial 0 finished with value: 187.04535978081637 and parameters: {'lr': 1.1843373531417966e-05, 'tau': 0.0026364498541427634, 'gamma': 0.23649797264153066}. Best is trial 0 with value: 187.04535978081637.


[Done] Total cputime: 232.55 s, system time: 59.32 s
AAMAS MASA Implementation 
mode: RLcontroller, rl_model_name: TD3, market_name: DJIA, topK: 10, dataDir: ./data, enable_controller: True, 
trade_pattern: 1 
period_mode: 1, num_epochs: 50, cov_lookback: 5, norm_method: sum, benchmark_algo: MASA-dc, trained_best_model_type: js_loss, pricePredModel: MA, 
is_enable_dynamic_risk_bound: True, risk_market: 0.001, risk_default: 0.017, cbf_gamma: 0.7, ars_trial: 10 
cur_datetime: 2024-05-12-12-57-10, res_dir: ./res\RLcontroller\TD3\DJIA-10\2024-05-12-12-57-10, tradeDays_per_year: 252, tradeDays_per_month: 21, seed_num: 1715533030, 
train_date_start: 2023-05-01 00:00:00, train_date_end: 2023-08-30 23:59:59, valid_date_start: 2023-08-31 00:00:00, valid_date_end: 2023-11-09 23:59:59, test_date_start: 2023-11-10 00:00:00, test_date_end: 2024-03-01 23:59:59, 
tech_indicator_input_lst: ['CHANGE'], 
otherRef_indicator_lst: ['MA-5', 'DAILYRETURNS-5'], enable_cov_features: False 
tmp_name: Cls3_RLcon

[I 2024-05-12 12:58:10,581] Trial 1 finished with value: 232.6024289633498 and parameters: {'lr': 0.005865187428995262, 'tau': 0.00023267325858175556, 'gamma': 0.23815583486884706}. Best is trial 1 with value: 232.6024289633498.


...

[I 2024-05-12 13:02:06,187] Trial 5 finished with value: 185.5858360739224 and parameters: {'lr': 1.276716580580807e-05, 'tau': 0.0010939080631721676, 'gamma': 0.5247044596286099}. Best is trial 1 with value: 232.6024289633498.


[Done] Total cputime: 227.84 s, system time: 58.37 s
AAMAS MASA Implementation 
mode: RLcontroller, rl_model_name: TD3, market_name: DJIA, topK: 10, dataDir: ./data, enable_controller: True, 
trade_pattern: 1 
period_mode: 1, num_epochs: 50, cov_lookback: 5, norm_method: sum, benchmark_algo: MASA-dc, trained_best_model_type: js_loss, pricePredModel: MA, 
is_enable_dynamic_risk_bound: True, risk_market: 0.001, risk_default: 0.017, cbf_gamma: 0.7, ars_trial: 10 
cur_datetime: 2024-05-12-13-02-06, res_dir: ./res\RLcontroller\TD3\DJIA-10\2024-05-12-13-02-06, tradeDays_per_year: 252, tradeDays_per_month: 21, seed_num: 1715533326, 
train_date_start: 2023-05-01 00:00:00, train_date_end: 2023-08-30 23:59:59, valid_date_start: 2023-08-31 00:00:00, valid_date_end: 2023-11-09 23:59:59, test_date_start: 2023-11-10 00:00:00, test_date_end: 2024-03-01 23:59:59, 
tech_indicator_input_lst: ['CHANGE'], 
otherRef_indicator_lst: ['MA-5', 'DAILYRETURNS-5'], enable_cov_features: False 
tmp_name: Cls3_RLcon

[W 2024-05-12 13:02:07,500] Trial 6 failed with parameters: {'lr': 0.0004182935228573875, 'tau': 0.027262013861071223, 'gamma': 0.42447247646063335} because of the following error: MemoryError((1000000, 1, 1251), dtype('float32')).
Traceback (most recent call last):
  File "c:\Users\MatthewWang\.conda\envs\vipmasa\lib\site-packages\optuna\study\_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\MatthewWang\AppData\Local\Temp\ipykernel_30372\1622800671.py", line 15, in objective
    my_entrance(sampled_params, config)
  File "C:\Users\MatthewWang\AppData\Local\Temp\ipykernel_30372\1635079604.py", line 29, in my_entrance
    RLcontroller(config=config)
  File "c:\Users\MatthewWang\Projects\vipmasa\VIP-APM-EIIE_RL\projects\masa\entrance.py", line 167, in RLcontroller
    time_usage = t.timeit(number=1)
  File "c:\Users\MatthewWang\.conda\envs\vipmasa\lib\timeit.py", line 178, in timeit
    timing = self.inner(it, self.timer)
  File "<timeit-src>", li

------------------------------
Mode: train, Ep: 1, Current epoch capital: 997705.8442367178, historical best captial (1 ep): 11.877502180333828 | solvable: 85, insolvable: 0 | step count: 0 | cputime cur: 1.58 s, avg: 1.58 s, system time cur: 0.35 s/ep, avg: 0.35 s/ep..


MemoryError: Unable to allocate 4.66 GiB for an array with shape (1000000, 1, 1251) and data type float32